# Clustering com Múltiplas Features

[Data Dicionary](https://sda.berkeley.edu/sdaweb/docs/scfcomb2019/DOC/hcbkx01.htm)

Na lição anterior, construímos um modelo K-Means para criar clusters de respondentes da Pesquisa de Finanças dos Consumidores. Criamos nossos clusters analisando apenas duas variáveis, mas há centenas de variáveis no conjunto de dados que não levamos em conta e que podem conter informações valiosas. Nesta lição, vamos examinar todas as variáveis e selecionar cinco para criar os clusters. Depois de construir nosso modelo e escolher um número apropriado de clusters, aprenderemos como visualizar clusters multidimensionais em um gráfico de dispersão 2D usando algo chamado análise de componentes principais (PCA).

In [ ]:
import pandas as pd
import plotly.express as px
from scipy.stats.mstats import trimmed_var
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Preparando Dados

## Importação

Passamos algum tempo na última lição focando em um subconjunto útil do SCF, e desta vez, vamos nos concentrar ainda mais. Um dos problemas persistentes que tivemos com este conjunto de dados é que ele inclui alguns outliers na forma de lares ultra-ricos. Isso não fez muita diferença na nossa última análise, mas pode causar problemas nesta lição, então vamos nos concentrar em famílias com patrimônio líquido abaixo de 2 milhões de dólares.

### Exercício:
Reescreva sua função `wrangle` da última lição para que ela retorne um DataFrame de lares cujo patrimônio líquido seja inferior a 2 milhões de dólares e que tenham sido recusados para crédito ou tenham temido ser negados crédito nos últimos 5 anos (veja `"TURNFEAR"`).

In [ ]:
def wrangle(filepath):
  df = pd.read_csv(filepath)

  mask = (df["TURNFEAR"] == 1) & (df["INCOME"] < 2e6)
  df = df[mask]

  return df

In [ ]:
df = wrangle('/content/SCFP2019.csv.gz')

print("df type:", type(df))
print("df shape:", df.shape)
df.head()

## Explorar

Nesta lição, queremos criar clusters utilizando mais de duas variáveis, mas quais das 351 variáveis devemos escolher? Muitas vezes, essa decisão será tomada por outra pessoa. Por exemplo, um interessado pode fornecer uma lista das variáveis que são mais importantes para ele. No entanto, se você não tiver essa limitação, outra maneira de escolher as melhores variáveis para clustering é determinar quais variáveis numéricas têm a maior **variância**. É isso que faremos aqui.

A __variância__ de uma variável numérica é uma medida de quão __*dispersos*__ os valores estão em relação à __*média*__ dessa variável.

Em termos simples, ela quantifica o quanto os dados variam ou se afastam da média.

Se a variância for alta, isso significa que os valores estão bem espalhados em torno da média, ou seja, há uma grande diferença entre os valores. Se a variância for baixa, os valores estão mais próximos da média, com pouca variação.

### Exercício:
Calcule a variância de todas as variáveis em `df` e crie uma Series `top_ten_var` com as 10 variáveis com a maior variância.

In [ ]:
# Calculate variance, get 10 largest features
top_ten_var = df.var().sort_values().tail(10)

print("top_ten_var type:", type(top_ten_var))
print("top_ten_var shape:", top_ten_var.shape)
top_ten_var

Como de costume, é mais difícil entender uma lista como essa do que seria se a visualizássemos, então vamos criar um gráfico.

### Exercício:
Use o Plotly Express para criar um gráfico de barras horizontais de `top_ten_var`. Certifique-se de rotular o eixo x como `"Variance"`, o eixo y como `"Feature"` e utilizar o título `"SCF: High Variance Features"`.

In [ ]:
# Create horizontal bar chart of `top_ten_var`
fig = px.bar(
    x = top_ten_var,
    y = top_ten_var.index,
    title = "SCF: High Variance Features"
)
fig.update_layout(xaxis_title="Variance", yaxis_title="Features")

fig.show()

Uma coisa que observamos ao longo deste projeto é que muitos dos indicadores de riqueza são altamente enviesados, com alguns lares outliers possuindo uma riqueza enorme. Esses outliers podem afetar nossa medida de variância. Vamos ver se isso acontece com uma das variáveis de `top_five_var`.

### Exercício:
Use o Plotly Express para criar um boxplot horizontal de `"NHNFIN"` para determinar se os valores estão enviesados. Certifique-se de rotular o eixo x como `"Value [$]"` e utilizar o título `"Distribution of Non-home, Non-Financial Assets"`.

In [ ]:
# Create a boxplot of `NHNFIN`
fig = px.box(
    data_frame=df,
    x="NHNFIN",
    title="Distribution of Non-home, Non-Financial Assets"
)
fig.update_layout(xaxis_title="$")
fig.show()

Uau! O conjunto de dados está imensamente enviesado à direita por causa dos grandes outliers no lado direito da distribuição. Embora já tenhamos excluído lares com um patrimônio líquido elevado com nossa função `wrangle`, a variância ainda está sendo distorcida por alguns outliers extremos.

A melhor maneira de lidar com isso é olhar para a **variância ajustada**, onde removemos valores extremos antes de calcular a variância. Podemos fazer isso usando a função `trimmed_variance` da biblioteca `SciPy`.

A função trimmed_var faz parte do módulo mstats (masked statistics) da SciPy, que é projetado para lidar com arrays mascarados (arrays que podem conter valores inválidos ou ausentes).

A função trimmed_var calcula a __variância aparada__ (trimmed variance) de um conjunto de dados.

__Variância Aparada (Trimmed Variance):__
É uma medida de dispersão que, antes de calcular a variância, remove uma certa porcentagem de valores extremos (tanto os mais baixos quanto os mais altos) dos dados. Esse procedimento ajuda a reduzir o impacto de outliers (valores atípicos) na estimativa da variância, proporcionando uma medida mais robusta da dispersão dos dados.

### Exercício:
Calcule a variância ajustada para as variáveis em `df`. Seus cálculos não devem incluir os 10% superiores e inferiores de observações. Em seguida, crie uma Series `top_ten_trim_var` com as 10 variáveis que possuem a maior variância ajustada.

In [ ]:
# Calculate trimmed variance
top_ten_trim_var = df.apply(trimmed_var).sort_values().tail(10)

print("top_ten_trim_var type:", type(top_ten_trim_var))
print("top_ten_trim_var shape:", top_ten_trim_var.shape)
top_ten_trim_var

Certo! Agora que temos um conjunto de números melhores, vamos criar outro gráfico de barras.

### Exercício:
Use o Plotly Express para criar um gráfico de barras horizontais de `top_ten_trim_var`. Certifique-se de rotular o eixo x como `"Trimmed Variance"`, o eixo y como `"Feature"` e utilizar o título `"SCF: High Variance Features"`.

In [ ]:
# Create horizontal bar chart of `top_ten_trim_var`
fig = px.bar(
    x=top_ten_trim_var,
    y=top_ten_trim_var.index,
    title="SCF: High Variance Features"
)
fig.update_layout(xaxis_title="Trimmed Variance", yaxis_title="Features")
fig.show()

Há três coisas a se notar neste gráfico.

Primeiro, as variâncias diminuíram bastante. No nosso gráfico anterior, o eixo x ia até 80 bilhões de dólares; este vai até 12 bilhões de dólares.

Em segundo lugar, as 10 principais variáveis mudaram um pouco. Todas as variáveis relacionadas à propriedade de negócios (`"...BUS"`) desapareceram.

Por fim, podemos ver que existem grandes diferenças na variância de uma variável para outra.

Por exemplo, a variância de `"WAGEINC"` é em torno de 500 milhões de dólares, enquanto a variância de `"ASSET"` é quase 12 bilhões de dólares. Em outras palavras, essas variáveis têm escalas completamente diferentes. Isso é algo que precisaremos abordar antes de podermos formar bons clusters.

### Exercício:
Gere uma lista chamada `high_var_cols` com os nomes das cinco variáveis com a maior variância ajustada.

In [ ]:
high_var_cols = top_ten_trim_var.tail(5).index.to_list()

print("high_var_cols type:", type(high_var_cols))
print("high_var_cols len:", len(high_var_cols))
high_var_cols

## Dividir

Agora que temos nossos dados em um formato que podemos usar, podemos seguir os passos que usamos anteriormente para construir um modelo, começando com uma matriz de características.

### Exercício:
Crie a matriz de características `X`. Ela deve conter as cinco colunas da lista `high_var_cols`.

In [ ]:
X = df[high_var_cols]

print("X type:", type(X))
print("X shape:", X.shape)
X.head()

# Construindo Model

## Iterar

Durante nossa EDA, vimos que tínhamos um problema de escala entre nossas variáveis. Esse problema pode dificultar o agrupamento dos dados, então precisaremos corrigi-lo para ajudar em nossa análise.

Uma estratégia que podemos usar é a **padronização**, um método estatístico para colocar todas as variáveis em um conjunto de dados na mesma escala.

Vamos explorar como isso funciona aqui. Mais tarde, o incorporaremos ao nosso pipeline do modelo.

### Exercício:
Crie um DataFrame chamado `X_summary` com a média e o desvio padrão de todas as variáveis em `X`.

In [ ]:
X_summary = X.aggregate(["mean","std"]).astype(int)

print("X_summary type:", type(X_summary))
print("X_summary shape:", X_summary.shape)
X_summary

Essas são as informações de que precisamos para padronizar nossos dados, então vamos colocá-las em prática.

A __*standardização*__ é uma técnica fundamental em análise de dados, especialmente quando se trabalha com métodos que são sensíveis à escala das variáveis, como o k-means clustering e muitos algoritmos de aprendizado de máquina.

__O que é Standardização?__

A standardização transforma as variáveis de modo que tenham média zero e desvio padrão um. Isso é feito subtraindo a média da variável e dividindo pelo desvio padrão.

__Benefícios da Standardização__

- Igualdade de Escala: Todas as variáveis estarão na mesma escala, o que é essencial para algoritmos que utilizam distâncias, como k-means e PCA (análise de componentes principais).

- Melhor Convergência: Alguns algoritmos, como gradientes, convergem mais rapidamente quando as características estão em escalas semelhantes.

- Interpretação dos Coeficientes: Em modelos lineares, a standardização ajuda na interpretação dos coeficientes.

__Conclusão:__
A standardização é uma etapa crucial no pré-processamento de dados, especialmente quando se trata de análise e modelagem. Incorporá-la em seu pipeline de machine learning não só melhora a performance do modelo, mas também facilita a interpretação dos resultados.

### Exercício:
Crie um transformador `StandardScaler`, use-o para transformar os dados em `X` e, em seguida, coloque os dados transformados em um DataFrame chamado `X_scaled`.

In [ ]:
# Instantiate transformer
ss = StandardScaler()

# Transform `X`
X_scaled_data = ss.fit_transform(X)

# Put `X_scaled_data` into DataFrame
X_scaled = pd.DataFrame(X_scaled_data, columns=X.columns)

print("X_scaled type:", type(X_scaled))
print("X_scaled shape:", X_scaled.shape)
X_scaled.head()

Como você pode ver, todas as cinco variáveis agora usam a mesma escala. Mas, apenas para garantir, vamos dar uma olhada em suas médias e desvios padrão.

### Exercício:
Crie um DataFrame chamado `X_scaled_summary` com a média e o desvio padrão de todas as variáveis em `X_scaled`.

In [ ]:
X_scaled_summary = X_scaled.aggregate(["mean","std"]).astype(int)

print("X_scaled_summary type:", type(X_scaled_summary))
print("X_scaled_summary shape:", X_scaled_summary.shape)
X_scaled_summary

E é assim que deve ficar. Lembre-se de que a padronização leva todas as variáveis e as escala para que todas tenham uma média de 0 e um desvio padrão de 1.

Agora que podemos comparar todos os nossos dados na mesma escala, podemos começar a formar os clusters. Assim como fizemos da última vez, precisamos descobrir quantos clusters devemos ter.

### Exercício:
Use um loop `for` para construir e treinar um modelo K-Means em que `n_clusters` varia de 2 a 12 (inclusivo). Seu modelo deve incluir um `StandardScaler`. Cada vez que um modelo é treinado, calcule a inércia e adicione-a à lista `inertia_errors`; em seguida, calcule a pontuação de silhueta e adicione-a à lista `silhouette_scores`.

In [ ]:
n_clusters = range(2,13)
inertia_errors = []
silhouette_scores = []

# Add `for` loop to train model and calculate inertia, silhouette score.
for k in n_clusters:
  model = make_pipeline(
      StandardScaler(),
      KMeans(n_clusters=k, random_state=42)
  )
  model.fit(X)
  inertia_errors.append(model.named_steps["kmeans"].inertia_)
  silhouette_scores.append(
      silhouette_score(X, model.named_steps["kmeans"].labels_)
  )



print("inertia_errors type:", type(inertia_errors))
print("inertia_errors len:", len(inertia_errors))
print("Inertia:", inertia_errors)
print()
print("silhouette_scores type:", type(silhouette_scores))
print("silhouette_scores len:", len(silhouette_scores))
print("Silhouette Scores:", silhouette_scores)

Assim como da última vez, vamos criar um gráfico de cotovelo para ver quantos clusters devemos usar.

### Exercício:
Use o Plotly Express para criar um gráfico de linhas que mostre os valores de `inertia_errors` em função de `n_clusters`. Certifique-se de rotular seu eixo x como `"Number of Clusters"`, seu eixo y como `"Inertia"` e utilizar o título `"K-Means Model: Inertia vs Number of Clusters"`.

In [ ]:
# Create line plot of `inertia_errors` vs `n_clusters`
fig = px.line(
    x=n_clusters,
    y=inertia_errors,
    title="K-Means Model: Inertia vs Number of Clusters"
)
fig.update_layout(xaxis_title="Number of Clusters", yaxis_title="Inertia")

fig.show()

Você pode ver que a linha começa a se achatar em torno de 4 ou 5 clusters.

**Nota:** Acabamos usando 5 clusters da última vez também, mas isso se deve ao fato de estarmos trabalhando com dados muito semelhantes. 5 clusters nem sempre será a escolha certa para esse tipo de análise, como veremos a seguir.

Vamos criar outro gráfico de linhas com base nas pontuações de silhueta.

### Exercício:
Use o Plotly Express para criar um gráfico de linhas que mostre os valores de `silhouette_scores` em função de `n_clusters`. Certifique-se de rotular seu eixo x como `"Number of Clusters"`, seu eixo y como `"Silhouette Score"` e utilizar o título `"K-Means Model: Silhouette Score vs Number of Clusters"`.

In [ ]:
# Create a line plot of `silhouette_scores` vs `n_clusters`
fig = px.line(
    x=n_clusters,
    y=silhouette_scores,
    title="K-Means Model: Silhouette Score vs Number of Clusters"
)
fig.update_layout(xaxis_title="Number of Clusters", yaxis_title="Silhouette Score")
fig.show()

Este gráfico é um pouco menos direto, mas podemos ver que as melhores pontuações de silhueta ocorrem quando há 3 ou 4 clusters.

Juntando as informações deste gráfico com o nosso gráfico de inércia, parece que a melhor configuração para `n_clusters` será 4.

### Exercício:
Construa e treine um novo modelo K-Means chamado `final_model`. Use as informações que você obteve a partir dos dois gráficos acima para definir um valor apropriado para o argumento `n_clusters`. Assim que você tiver construído e treinado seu modelo, envie-o para avaliação.

In [ ]:
from typing import final
# Build model
final_model = make_pipeline(
    StandardScaler(),
    KMeans(n_clusters=4, random_state=42)
)

# Fit model to data
final_model.fit(X)


# Comunicar Resultados


É hora de informar a todos como as coisas terminaram. Vamos começar pegando os rótulos.

### Exercício:
Extraia os rótulos que seu `final_model` criou durante o treinamento e atribua-os à variável `labels`.

In [ ]:
labels = final_model.named_steps["kmeans"].labels_

print("labels type:", type(labels))
print("labels len:", len(labels))
print(labels[:5])

Vamos criar um novo DataFrame para trabalharmos na visualização.

### Exercício:
Crie um DataFrame `xgb` que contenha os valores médios das características em `X` para cada um dos clusters em seu `final_model`.

In [ ]:
xgb = X.groupby(labels).mean()

print("xgb type:", type(xgb))
print("xgb shape:", xgb.shape)
xgb

Agora que temos um DataFrame, vamos fazer um gráfico de barras e ver como nossos clusters diferem.

### Exercício:
Use o Plotly Express para criar um gráfico de barras lado a lado a partir de `xgb` que mostre a média das características em `X` para cada um dos clusters em seu `final_model`. Certifique-se de rotular o eixo x como `"Cluster"`, o eixo y como `"Value [$]"` e utilizar o título `"Mean Household Finances by Cluster"`.

In [ ]:
# Create side-by-side bar chart of `xgb`
fig = px.bar(
  xgb,
  barmode="group",
  title="Mean Household Finances by Cluster"
)
fig.update_layout(xaxis_title="Cluster",yaxis_title="Value [$]")
fig.show()

Lembre-se de que nossos clusters são baseados parcialmente no `NETWORTH`, o que significa que os lares no cluster 0 têm a menor riqueza líquida, enquanto os lares no cluster 2 têm a maior. Com base nisso, há algumas coisas interessantes para explorar aqui.

Primeiro, observe a variável `DEBT`.

Você poderia pensar que ela aumentaria conforme a riqueza líquida aumenta, mas não é o que acontece.

A menor quantidade de dívida é carregada pelos lares do cluster 2, mesmo que o valor de suas casas (mostrado em verde) seja aproximadamente o mesmo. Você não pode *realmente* dizer com esses dados o que está acontecendo, mas uma possibilidade é que as pessoas do cluster 2 tenham dinheiro suficiente para quitar suas dívidas, mas não o suficiente para alavancar o que possuem em dívidas adicionais. Por outro lado, as pessoas do cluster 3 podem não precisar se preocupar em carregar dívidas porque sua riqueza líquida é tão alta.

Finalmente, como começamos este projeto analisando os valores de casas, observe a relação entre `DEBT` e `HOUSES`. O valor da dívida para as pessoas no cluster 0 é maior que o valor de suas casas, sugerindo que a maior parte da dívida que essas pessoas carregam está atrelada a suas hipotecas — se é que possuem uma casa. Em contraste com os outros três clusters: o valor da dívida de todos os outros é menor que o valor de suas casas.

Então, tudo isso é bem interessante, mas é diferente do que fizemos da última vez, certo? Neste ponto da última lição, fizemos um gráfico de dispersão. Essa foi uma tarefa simples porque trabalhamos apenas com duas características, então podíamos plotar os pontos de dados em duas dimensões. Mas agora `X` tem cinco dimensões! Como podemos plotar isso para dar aos stakeholders uma ideia de nossos clusters?

Como estamos trabalhando com uma tela de computador, não temos muito espaço em relação ao número de dimensões que podemos usar: precisamos que sejam duas. Portanto, se formos fazer algo parecido com o gráfico de dispersão que fizemos antes, precisaremos transformar nossos dados de 5 dimensões em algo que possamos visualizar em 2 dimensões.

### Principal Component Analysis (PCA)

É uma técnica estatística usada para simplificar um conjunto de dados de alta dimensionalidade, reduzindo-o a um número menor de componentes principais que ainda capturam a maior parte da variância presente nos dados originais.

Essencialmente, o PCA transforma as variáveis originais em um novo conjunto de variáveis não correlacionadas chamadas componentes principais.

__Objetivos do PCA:__

- Redução de Dimensionalidade: Reduz o número de variáveis mantendo a maior quantidade possível de informação.
Visualização: Facilita a visualização de dados complexos em 2D ou 3D.

- Remoção de Multicolinearidade: Elimina redundâncias entre variáveis correlacionadas.

- Melhoria de Performance: Reduz o tempo de treinamento de modelos de machine learning, especialmente em conjuntos de dados com muitas características.

__Como Funciona o PCA?__

O PCA identifica as direções (componentes principais) em que os dados variam mais. Cada componente principal é uma combinação linear das variáveis originais e é ortogonal aos demais componentes. A variância dos dados ao longo de cada componente principal decresce à medida que avançamos nos componentes.

### Exercício:
Crie um transformador `PCA`, use-o para reduzir a dimensionalidade dos dados em `X` para 2 e, em seguida, coloque os dados transformados em um DataFrame chamado `X_pca`. As colunas de `X_pca` devem ser nomeadas como `"PC1"` e `"PC2"`.

In [ ]:
# Instantiate transformer
pca = PCA(n_components=2, random_state=42)

# Transform `X`
X_t = pca.fit_transform(X)

# Put `X_t` into DataFrame
X_pca = pd.DataFrame(X_t, columns=["PC1","PC2"])

print("X_pca type:", type(X_pca))
print("X_pca shape:", X_pca.shape)
X_pca.head()

Então é isso: nossas cinco dimensões foram reduzidas para duas. Vamos fazer um gráfico de dispersão e ver o que conseguimos.

### Exercício:
Use o plotly express para criar um gráfico de dispersão de `X_pca` usando o seaborn. Certifique-se de colorir os pontos de dados usando os rótulos gerados pelo seu `final_model`. Rotule o eixo x como `"PC1"`, o eixo y como `"PC2"` e use o título `"Representação PCA dos Clusters"`.

In [ ]:
# Create scatter plot of `PC2` vs `PC1`
fig = px.scatter(
    data_frame = X_pca,
    x = "PC1",
    y = "PC2",
    title="Representação PCA dos Clusters"
)
fig.show()

**Nota:** Você pode frequentemente melhorar o desempenho do PCA padronizando seus dados primeiro.
Experimente incluir um `StandardScaler` em sua transformação de `X`.

Como isso muda os clusters em seu gráfico de dispersão?

Uma limitação deste gráfico é que é difícil explicar o que os eixos representam aqui. Na verdade, ambos são uma combinação das cinco características que originalmente tínhamos em `X`, o que significa que isso é bastante abstrato. Ainda assim, é a melhor maneira que temos para mostrar o máximo de informações possível como uma ferramenta explicativa para pessoas fora da comunidade de ciência de dados.

Então, o que esse gráfico significa? Significa que fizemos quatro clusters agrupados de forma compacta que compartilham algumas características-chave. Se estivéssemos apresentando isso a um grupo de partes interessadas, poderia ser útil mostrar esse gráfico primeiro como uma espécie de aquecimento, já que a maioria das pessoas entende como um objeto bidimensional funciona. Depois, poderíamos passar para uma análise mais sutil dos dados.

Apenas algo a ter em mente enquanto você continua sua jornada na ciência de dados.

# Dashboard Interativo

Na última lição, construímos um modelo com base nas características de maior variância em nosso conjunto de dados e criamos várias visualizações para comunicar nossos resultados. Nesta lição, vamos combinar todos esses elementos em uma aplicação web dinâmica que permitirá aos usuários escolher suas próprias características, construir um modelo e avaliar seu desempenho por meio de uma interface gráfica do usuário. Em outras palavras, você criará uma ferramenta que permitirá que qualquer pessoa construa um modelo sem precisar de código.

In [ ]:
!pip install dash jupyter-dash flask

In [2]:
import pandas as pd
import plotly.express as px

from dash import Input, Output, dcc, html
from jupyter_dash import JupyterDash
from flask import Flask

from scipy.stats.mstats import trimmed_var
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# configurar o Dash em ambientes Jupyter que usam um proxy
# JupyterDash.infer_jupyter_proxy_config()

# Preparando Dados

Como sempre, começaremos trazendo nossos dados para o projeto usando uma função `wrangle`.

## Importação

### Exercício 1:
Complete a função `wrangle` abaixo, usando a docstring como guia. Em seguida, utilize sua função para ler o arquivo `"data/SCFP2019.csv.gz"` em um DataFrame.

In [3]:
def wrangle(filepath):
    """Read SCF data file into ``DataFrame``.

    Returns only credit fearful households whose net worth is less than $2 million.

    Parameters
    ----------
    filepath : str
        Location of CSV file.
    """
    df = pd.read_csv(filepath)
    mask = (df["TURNFEAR"] == 1) & (df["NETWORTH"] < 2e6)

    df = df[mask]
    return df

In [4]:
df = wrangle('/content/SCFP2019.csv.gz')

print("df type:", type(df))
print("df shape:", df.shape)
df.head()

df type: <class 'pandas.core.frame.DataFrame'>
df shape: (4418, 351)


,YY1,Y1,WGT,HHSEX,AGE,AGECL,EDUC,EDCL,MARRIED,KIDS,...,NWCAT,INCCAT,ASSETCAT,NINCCAT,NINC2CAT,NWPCTLECAT,INCPCTLECAT,NINCPCTLECAT,INCQRTCAT,NINCQRTCAT
5,2,21,3790.476607,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2
6,2,22,3798.868505,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,3,2,2
7,2,23,3799.468393,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2
8,2,24,3788.076005,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2
9,2,25,3793.066589,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2


# Contruir Dashboard

É hora de criar o aplicativo! Existem muitos passos a seguir aqui, mas, no final, você terá feito um painel interativo! Começaremos com o layout.

## Application Layout

Primeiro, instancie a aplicação.

### Exercício 2:
Instancie uma aplicação `JupyterDash` e atribua-a à variável chamada `app`.

In [5]:
server = Flask(__name__)
app = JupyterDash(__name__, server=server)

/usr/local/lib/python3.10/dist-packages/dash/dash.py:585: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Então, vamos dar alguns rótulos ao aplicativo.

### Exercício 3:
Comece a construir o layout do seu `app` criando um objeto `Div` que tenha dois objetos filhos: um cabeçalho `H1` que exibe `"Survey of Consumer Finances"` e um cabeçalho `H2` que exibe `"High Variance Features"`.

**Nota:** Vamos construir o layout da nossa aplicação de forma iterativa. Portanto, esteja preparado para voltar a este bloco de código várias vezes à medida que adicionamos recursos.

In [13]:
app.layout = html.Div(
    [
        html.H1("Survey of Consumer Finances"),
        html.H2("High Variance Features"),
        dcc.Graph(figure=serve_bar_chart(),id="bar-chart"),
        dcc.RadioItems(
            options=[
                {"label": "Trimmed", "value": True},
                {"label": "Not Trimmed", "value": False}
                ],
            value=True,
            id="trim-button"
        ),
        html.H2("K-means Clustering"),
        html.H3("Number of Clusters (k)"),
        dcc.Slider(min=2,max=12,value=2,step=1,id="k-slider"),
        html.Div(id="metrics"),
        dcc.Graph(id="pct-scatter")

    ]
)

Eventualmente, o aplicativo que faremos terá várias partes interativas. Começaremos com um gráfico de barras.

## Variance Bar Chart

Não importa quão bem projetado o gráfico possa ser, ele não aparecerá no aplicativo a menos que o adicionemos ao painel como um objeto primeiro.

### Exercício 4:
Adicione um objeto `Graph` ao layout da sua aplicação. Certifique-se de atribuir a ele o id `"bar-chart"`.

Assim como fizemos da última vez, precisamos recuperar os recursos com a maior variância.

### Exercício 5:
Crie uma função `get_high_var_features` que retorne os cinco recursos com maior variância em um DataFrame. Use a docstring como guia.

In [6]:
def get_high_var_features(trimmed=True, return_feat_names=False):
    """Returns the five highest-variance features of ``df``.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.

    return_feat_names : bool, default=False
        If ``True``, returns feature names as a ``list``. If ``False``
        returns ``Series``, where index is feature names and values are
        variances.
    """
    if trimmed:
      top_five_features = df.apply(trimmed_var).sort_values().tail(5)
    else:
      top_five_features = df.var().sort_values().tail(5)


    if return_feat_names:
      top_five_features = top_five_features.index.to_list()

    return top_five_features

In [ ]:
get_high_var_features(trimmed=True, return_feat_names=True)

Agora que temos nossas cinco principais características, podemos usar uma função para retorná-las em um gráfico de barras.

### Exercício 6:
Crie uma função `serve_bar_chart` que retorne um gráfico de barras do Plotly Express com as cinco características de maior variância. Você deve usar `get_high_var_features` como uma função auxiliar. Siga a docstring como guia.

In [7]:
@app.callback(
    Output("bar-chart", "figure"),
    Input("trim-button", "value")
)
def serve_bar_chart(trimmed=True):

    """Returns a horizontal bar chart of five highest-variance features.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.
    """
    top_five_features = get_high_var_features(trimmed=trimmed, return_feat_names=False)

    fig = px.bar(
        x=top_five_features,
        y=top_five_features.index,
    )
    fig.update_layout(xaxis_title="Variance",yaxis_title="Features")

    return fig

In [ ]:
serve_bar_chart(False)

Agora, adicione o gráfico real ao aplicativo.

### Exercício 7:
Use sua função `serve_bar_chart` para adicionar um gráfico de barras ao `"bar-chart"`.

O que fizemos até agora não foi muito diferente de outras visualizações que construímos no passado. A maioria desses gráficos tem sido estática, mas este será interativo. Vamos adicionar um botão de opção para dar às pessoas algo com que brincar.

### Exercício 8:
Adicione um botão de opção ao layout da sua aplicação. Ele deve ter duas opções: `"trimmed"` (que possui o valor `True`) e `"not trimmed"` (que possui o valor `False`). Certifique-se de atribuir a ele o id `"trim-button"`.

Agora que temos o código para criar nosso gráfico de barras, um espaço em nosso aplicativo para colocá-lo e um botão para manipulá-lo, vamos conectar os três elementos.

### Exercício 9:
Adicione um decorador de callback à sua função `serve_bar_chart`. A entrada do callback deve ser o valor retornado por `"trim-button"` e a saída deve ser direcionada a `"bar-chart"`.

Quando estiver satisfeito com seu gráfico de barras e botões de opção, role até o final desta página e execute o último bloco de código para ver seu trabalho em ação!

## K-means Slider and Metrics

Ok, agora nosso aplicativo tem um botão de opção, mas isso é apenas uma interação para o visualizador. Os botões são divertidos, mas e se fizéssemos um controle deslizante para ajudar as pessoas a ver o que significa a mudança no número de clusters? Vamos fazer isso!

Novamente, comece adicionando alguns objetos ao layout.

### Exercício 10:
Adicione dois objetos de texto ao layout da sua aplicação: um cabeçalho `H2` que exibe `"K-means Clustering"` e um cabeçalho `H3` que exibe `"Number of Clusters (k)"`.

Agora adicione um slider

### Exercício 11:
Adicione um controle deslizante ao layout da sua aplicação. Ele deve variar de `2` a `12`. Certifique-se de atribuir a ele o id `"k-slider"`.

E adicione a coisa toda para o app.

### Exercício 12:
Adicione um objeto `Div` ao layout da sua aplicação. Certifique-se de atribuir a ele o id `"metrics"`.

Então, agora temos um gráfico de barras que muda com um botão de opção e um controle deslizante que muda... bem, ainda nada. Vamos dar a ele um modelo para trabalhar.

### Exercício 13:
Crie uma função `get_model_metrics` que construa, treine e avalie o modelo `KMeans`. Use a docstring como guia. Observe que, assim como o modelo que você fez na última lição, seu modelo aqui deve ser um pipeline que inclui um `StandardScaler`. Quando terminar, envie sua função para o avaliador.

In [8]:
def get_model_metrics(trimmed=True, k=2, return_metrics=False):

    """Build ``KMeans`` model based on five highest-variance features in ``df``.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.

    k : int, default=2
        Number of clusters.

    return_metrics : bool, default=False
        If ``False`` returns ``KMeans`` model. If ``True`` returns ``dict``
        with inertia and silhouette score.

    """
    features = get_high_var_features(trimmed=trimmed, return_feat_names=True)
    X = df[features]
    model = make_pipeline(
        StandardScaler(),
        KMeans(n_clusters=k,random_state=42)
    )
    model.fit(X)

    if return_metrics:
      i = model.named_steps["kmeans"].inertia_
      ss = silhouette_score(X, model.named_steps["kmeans"].labels_)
      metrics = {
          "inertia": round(i), "silhouette": round(ss,3)
      }
      return metrics

    return model

In [ ]:
get_model_metrics(trimmed=False, k=2, return_metrics=True)

Parte do que queremos que as pessoas possam fazer com o painel é ver como a inércia do modelo e a pontuação de silhueta mudam quando elas movimentam o controle deslizante, então vamos calcular esses números...

### Exercício 14:
Crie uma função `serve_metrics`. Ela deve usar sua função `get_model_metrics` para construir e obter as métricas de um modelo e, em seguida, retornar dois objetos: um cabeçalho `H3` com a inércia do modelo e outro cabeçalho `H3` com a pontuação de silhueta.

In [9]:
@app.callback(
    Output("metrics", "children"),
    Input("trim-button", "value"),
    Input("k-slider", "value")
)
def serve_metrics(trimmed=True,k=2):

    """Returns list of ``H3`` elements containing inertia and silhouette score
    for ``KMeans`` model.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.

    k : int, default=2
        Number of clusters.
    """
    metrics = get_model_metrics(trimmed=trimmed, k=k, return_metrics=True)
    text = [
        html.H3(f"Inertia: {metrics['inertia']}"),
        html.H3(f"Silhouete: {metrics['silhouete']}")
    ]

    return text

... E adicione no app

### Exercício 15:
Adicione um decorador de callback à sua função `serve_metrics`. As entradas do callback devem ser os valores retornados por `"trim-button"` e `"k-slider"`, e a saída deve ser direcionada a `"metrics"`.

## PCA Scatter Plot

Acabamos de criar um controle deslizante que pode alterar os valores de inércia e silhueta, mas nem todos conseguirão entender o que esses números em mudança significam. Vamos fazer um gráfico de dispersão para ajudar nesse entendimento.

### Exercício 16:
Adicione um objeto `Graph` ao layout da sua aplicação. Certifique-se de atribuir a ele o id `"pca-scatter"`.

Assim como fizemos com o gráfico de barras, precisamos obter as cinco características de maior variância dos dados, então vamos começar por isso.

### Exercício 17:
Crie uma função `get_pca_labels` que faça um subconjunto de um DataFrame para suas cinco características de maior variância, reduza essas características para duas dimensões usando `PCA` e retorne um novo DataFrame com três colunas: `"PC1"`, `"PC2"` e `"labels"`. Esta última coluna deve ser os rótulos determinados por um modelo `KMeans`. Sua função deve usar `get_high_var_features` e `get_model_metrics` como auxiliares. Consulte a docstring para obter orientações.

In [21]:
def get_pca_labels(trimmed=True,k=2):

    """
    ``KMeans`` labels.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.

    k : int, default=2
        Number of clusters.
    """
    features = get_high_var_features(trimmed=trimmed, return_feat_names=True)
    X = df[features]
    pca = PCA(n_components=2, random_state=42)
    X_t = pca.fit_transform(X)
    X_pca = pd.DataFrame(X_t, columns=["PC1","PC2"])

    model = get_model_metrics(trimmed=trimmed, k=k, return_metrics=False)
    X_pca["Labels"] = model.named_steps["kmeans"].labels_.astype(str)
    X_pca.sort_values("Labels",inplace=True)

    return X_pca

In [ ]:
get_pca_labels().info()

Agora podemos usar essas cinco características para criar o gráfico de dispersão real.

### Exercício 18:
Crie uma função `serve_scatter_plot` que crie um gráfico de dispersão 2D dos dados usados para treinar um modelo `KMeans`, juntamente com clusters codificados por cores. Use `get_pca_labels` como auxiliar. Consulte a docstring para obter orientações.

In [25]:
@app.callback(
    Output("pca-scatter", "figure"),
    Input("trim-button", "value"),
    Input("k-slider", "value")
)
def serve_scatter_plot(trimmed=True,k=2):

    """Build 2D scatter plot of ``df`` with ``KMeans`` labels.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.

    k : int, default=2
        Number of clusters.
    """
    fig = px.scatter(
        data_frame=get_pca_labels(trimmed=trimmed, k=k),
        x="PC1",
        y="PC2",
        color="Labels"
    )

    return fig

In [ ]:
serve_scatter_plot()

Novamente, finalizamos adicionando algum código para fazer os elementos interativos do nosso aplicativo realmente funcionarem.

### Exercício 19:
Adicione um decorador de callback à sua função `serve_scatter_plot`. As entradas do callback devem ser os valores retornados por `"trim-button"` e `"k-slider"`, e a saída deve ser direcionada a `"pca-scatter"`.

## Application Deployment

Assim que você se sentir bem com todo o trabalho que acabamos de fazer, execute a célula e veja o aplicativo ganhar vida!

### Exercício 20:
Rode a célula abaixo para fazer o deploy do seu app! 😎

In [ ]:
app.run_server(port=8050, mode='external')